In [1]:
import os, glob, json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import collections
import cv2
import tqdm
from PIL import Image
import torch

import torchvision.models

In [2]:
import sys
sys.path.append('/home/users/jsoelter/Code/ChestImageAI/utils/')
sys.path.append('/home/users/jsoelter/Code/big_transfer/')

import data_loader, evaluations, model_setup

In [20]:
original_model = torchvision.models.densenet121(pretrained=True)
backbone = original_model.features
features = self.backbone(x)
out = F.relu(features, inplace=True)
out = F.adaptive_avg_pool2d(out, (1, 1))
out = torch.flatten(out, 1)

In [ ]:
features = self.backbone(x)

In [6]:
ls ~/models/BiT

BiT-M-R101x1.npz  BiT-M-R50x1.npz    BiT-M-R50x3.npz
BiT-M-R101x3.npz  BiT-M-R50x1.npz.1


In [6]:
wget https://storage.googleapis.com/bit_models/BiT-M-R50x1.{npz|h5}

Signature: F.relu(input, inplace=False)
Docstring:
relu(input, inplace=False) -> Tensor

Applies the rectified linear unit function element-wise. See
:class:`~torch.nn.ReLU` for more details.
File:      ~/miniconda3/lib/python3.7/site-packages/torch/nn/functional.py
Type:      function


In [5]:
import torch.nn.functional as F

In [ ]:
class Backbone(torch.nn.Module):

    def __init__(self, name, flat=True):
        
        if name == 'densenet121':
            model = torchvision.models.densenet121(pretrained= (pretrained == 'imagenet'))
            self.backbone = model.features
            self.flatten = torch.nn.ReLU
            

In [20]:
model.module.body.block4.unit03.conv3.out_channels

6144

In [35]:
from bit_pytorch.models import ResNetV2, KNOWN_MODELS as BiTModels
from collections import OrderedDict
from torch import nn

class ResNetV2_feature_extractor(ResNetV2):
    
    def __init__(self, block_units, width_factor, head_size=21843, zero_head=False):
        super().__init__(block_units, width_factor, head_size=21843, zero_head=False)
        self.pre_head = nn.Sequential(OrderedDict([
            ('gn', nn.GroupNorm(32, 2048*width_factor)),
            ('relu', nn.ReLU(inplace=True)),
            ('avg', nn.AdaptiveAvgPool2d(output_size=1))
        ]))
        
    def forward(self, x):
        x = self.pre_head(self.body(self.root(x)))
        return x



class BiT_MetaMixin(torch.nn.Module):
    
    def __init__(self, num_meta_data, backbone):
        super().__init__()

        self.backbone = backbone
        self.classifier =nn.Conv2d(self.backbone.body.block4.unit03.conv3.out_channels+num_meta_data, 1, kernel_size=(1, 1), stride=(1, 1))
        

    def forward(self, im, meta_data):
        
        img_feat = self.backbone(im)
        feat = torch.cat([img_feat, meta_data], 1)      
        pred = self.classifier(feat)
        
        return pred
    
m = BiT_MetaMixin(1, ResNetV2_feature_extractor([3, 4, 6, 3], 3, head_size=1, zero_head=True))

### Model Setup

In [3]:
model_dict =  dict(
    architecture = 'BiT-M-R50x3',
    #architecture = 'densenet121',
    num_classes = 5,
    pretrained = None, #'imagenet', #'/home/users/jsoelter/models/chexpert/fullmeta_503/step05000.pt',
    fresh_head_weights = False
)

In [5]:
model_out = '/home/users/jsoelter/models/chexpert/densenet/test'

if not os.path.exists(model_out):
    os.makedirs(model_out)

model = model_setup.instantiate_model(**model_dict)

saved_models = glob.glob(os.path.join(model_out, 'step*.pt'))
if not saved_models:
    checkpoint = None
    ledger = collections.defaultdict(list)
    ledger['model'] = model_dict
    step = 0
else:
    last_model = np.sort(saved_models)[-1]
    print(f"Resume training for saved model '{last_model}'")
    checkpoint = torch.load(last_model, map_location="cpu")
    re_keyed = {k.split('module.')[-1]: v for k, v in checkpoint['model'].items()}
    model.load_state_dict(re_keyed)
    
    ledger = json.load(open(os.path.join(model_out, 'train_ledger.json')))
    step = checkpoint["step"]

    
# Lets cuDNN benchmark conv implementations and choose the fastest.
# Only good if sizes stay the same within the main loop!
torch.backends.cudnn.benchmark = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.nn.DataParallel(model)
model = model.to(device)

### Data Setup

In [5]:
data_setup = dict(
    include_meta = [],
    #include_meta = ['Sex', 'AP/PA', 'Frontal/Lateral'],
    label_value_map = {
       0: 0.01,
       'nan': 0.05,
       -1.: 0.5,
       1: 0.98
    },
    fill_hierachy = {
        #'Enlarged Cardiomediastinum': ['Cardiomegaly'],
        #'Consolidation': ['Pneumonia'],
        #'Lung Opacity': ['Edema', 'Pneumonia', 'Consolidation', 'Lung Lesion', 'Atelectasis']
    },
    labels = ['Cardiomegaly', 'Edema',  'Consolidation', 'Atelectasis', 'Pleural Effusion'],
    subset = {}, # Define subsetting of data
)

transforms = [
    ('ToPILImage', {}),
    ('Resize', {
        'size': 136 #smaller edege mapped to x
    }),
    #('Resize', {
    #    'size': 544
    #}),
    ('RandomRotation', {
        'degrees': 5
    }),    
    ('RandomCrop', {
        'size': (128, 128)
    }),
    ('ToTensor', {}),
    ('Normalize', {
        'mean': [0.485, 0.456, 0.406], #'mean': (0.5, 0.5, 0.5),
        'std': [0.229, 0.224, 0.225]  #(0.5, 0.5, 0.5)
    }),
]

In [6]:
preprocess = data_loader.transform_pipeline_from_dict(transforms)

data = data_loader.ChexpertData('CheXpert-v1.0/train.csv', transform=preprocess, **data_setup)
internal_valid_data, train_data = torch.utils.data.random_split(data, [1000, len(data)-1000], generator=torch.Generator().manual_seed(42))
external_valid_data = data_loader.ChexpertData('CheXpert-v1.0/valid.csv', transform=preprocess, 
                                               labels=data_setup['labels'], include_meta=data_setup['include_meta'])

Removed 0 entries
Removed 0 entries


In [16]:
data.meta_df.shape

(223414, 19)

In [15]:
data.meta_df.Path.str.split('/').str.get(-3).nunique()

64540

In [ ]:
real_batch_size = 64#128
batch_split = 1 #8 #number of forward pathes before optimization is performed 

train_loader = torch.utils.data.DataLoader(train_data, batch_size=int(real_batch_size/batch_split), num_workers=8, shuffle=True, drop_last=False)

valid_int_loader = torch.utils.data.DataLoader(internal_valid_data, batch_size=int(real_batch_size/batch_split), num_workers=8, shuffle=True)
valid_ext_loader = torch.utils.data.DataLoader(external_valid_data, batch_size=16, num_workers=8)

len(train_loader)/batch_split

### Optimizer Setup

In [ ]:
opt = 'Adam'#'SGD'
opt_param = dict(
    lr = 1E-4,
    #momentum=0.9
)

In [ ]:
#optim = torch.optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
#optim = torch.optim.SGD([p for n,p in model.named_parameters() if 'head' in n], lr=0.003, momentum=0.9)
optim = getattr(torch.optim, opt)(model.parameters(), **opt_param)
if  checkpoint is not None:
    optim.load_state_dict(checkpoint["optim"])
else:
    optim.zero_grad()

In [ ]:
supports = [-1, 2*len(train_loader), int(3*len(train_loader)), int(4*len(train_loader))]#[3000, 7000, 9000, 10000]

### Loss

In [ ]:
train_on = None #['Cardiomegaly'] #,  'Consolidation' 'Cardiomegaly', 'Atelectasis', 'Pleural Effusion', 'Edema', 'Support Devices',  'AP/PA', 'Sex', 'Frontal/Lateral'] #,
train_cols = [i in train_on for i in data.targets] if train_on else None

In [ ]:
crit = model_setup.maskedBCE(train_cols, device)

### Initial errors

In [ ]:
preds, targets = evaluations.batch_prediction(model, valid_ext_loader)
print(f'AUC on {train_on}: {evaluations.eval_auc(preds, targets, train_cols):.3f}')
print(f'Crit on {train_on}: {evaluations.eval_crit(model, valid_int_loader, crit):.3f}')

### Training Loop

In [ ]:
eval_intervall = 50
save_intervall = 3500

In [ ]:
accum_steps = 0
batch_loss, batch_samples = 0, 0
lr = opt_param['lr']

train_setup = ledger.setdefault('train_setup', {})
train_setup[step] = {
    'transforms': transforms,
    'data_setup': data_setup,
    'optimizer': {
        'name': opt,
        'param': opt_param
    },
    'real_batch_size': real_batch_size,
    'batch_split': batch_split
}

while lr:
    for x, y in train_loader:
        
        _ = model.train()
        
        # Schedule sending to GPU(s)
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        # Update learning-rate, including stop training if over.
        lr = model_setup.get_lr(step, supports=supports, base_lr=opt_param['lr'])
        if lr is None: break
        for param_group in optim.param_groups:
            param_group["lr"] = lr

        logits = model(x)
        loss, n_samples = crit(logits, y)
        if loss != 0:
            # Accumulate grads
            (loss / batch_split / n_samples).backward()

        batch_loss += float(loss.data.cpu().numpy())  # Also ensures a sync point.
        batch_samples += n_samples.cpu().numpy()

        accum_steps += 1

        # Update params
        if accum_steps == batch_split:
            optim.step()
            optim.zero_grad()
            train_loss = batch_loss/batch_samples
            ledger['train_loss'].append(train_loss)
            batch_loss, batch_samples = 0, 0
            ledger['lr'].append(lr)
            step += 1
            accum_steps = 0
            
            # Evaluate 
            if (step % eval_intervall) == 0:
                preds, targets = evaluations.batch_prediction(model, valid_ext_loader)
                auc_selection = evaluations.eval_auc(preds, targets, train_cols)
                benchmark_cols = [i in ['Consolidation',  'Cardiomegaly', 'Atelectasis', 'Pleural Effusion', 'Edema'] for i in data.targets] 
                auc_bench = evaluations.eval_auc(preds, targets, benchmark_cols)
                ledger['external'].append((step-1, auc_selection))
                val = evaluations.eval_crit(model, valid_int_loader, crit)
                ledger['internal'].append((step-1, val))
                print(f'step {step} ->, train: {train_loss:.3f}, val internal : {val:.3f}, auc select: {auc_selection:.3f}, auc bench: {auc_bench:.3f}') # FULL: 

            if (step % save_intervall) == 0:
                torch.save({
                        "step": step,
                        "model": model.module.state_dict(),
                        "optim" : optim.state_dict(),
                    }, 
                    os.path.join(model_out, f'step{step:05d}.pt')
                )
                json.dump(ledger, open(os.path.join(model_out, 'train_ledger.json'), 'w'))

In [ ]:
torch.save({
        "step": step,
        "model": model.module.state_dict(),
        "optim" : optim.state_dict(),
    }, 
    os.path.join(model_out, f'step{step:05d}.pt')
)
json.dump(ledger, open(os.path.join(model_out, 'train_ledger.json'), 'w'))

In [ ]:
1+1

In [ ]:
ledger['model'] = model_dict

In [ ]:
torch.cuda.empty_cache()